In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

Mounted at /content/gdrive


In [ ]:
## I've only included movies/shows that got more than 50 votes. 
## That way it isn't a bunch of movies that aren't 'culturaly important'.
## It grabs the title, the plot, the year, and the director(s) of the movie. (the full date wasn't available on the search result)
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "lxml")
    movies = []
    for item in soup.find_all('div', class_='lister-item mode-advanced'):
        plot = item.find('p', class_='text-muted').find_next_sibling('p', class_='text-muted').get_text()
        if plot.__contains__('See full summary') or plot.__contains__('Add a Plot'):
            continue
        try:
          #This is my attempt to fix the case that there are multiple directors. Hopefully it doesn't suck.
          text = item.find('p', class_='text-muted').find_next_sibling('p', class_='text-muted').find_next_sibling('p').get_text()
          director = text[text.find(':')+2:text.find('|')].replace('\n', '')
        except:
          continue
        try:
            header = item.find('h3', class_='lister-item-header')
            year = header.find('span', class_='lister-item-year text-muted unbold').get_text()
            year = re.search(r'\d{4}', year).group()
        except:
            continue
        try:
            title = header.find('a').get_text()
        except:
            continue
        movies.append({'title': title, 'year': year, 'plot_summary': plot.replace('\n', ''), 'director': director})
    next_page = 'https://imdb.com' + soup.find('a', class_='lister-page-next next-page').get('href') if soup.find('a', class_='lister-page-next next-page') else None
    return movies, next_page

next_page = 'https://www.imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=1&ref_=adv_nxt'

results = []

while True:
    url = next_page
    print(f"Scraping: {url}")
    result_of_scrape = scrape_page(url)
    if not result_of_scrape[0]:
        break
    results.extend(result_of_scrape[0])
    next_page = result_of_scrape[1]


Scraping: https://www.imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=1&ref_=adv_nxt
Scraping: https://imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=251
Scraping: https://imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=501
Scraping: https://imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=751
Scraping: https://imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=1001
Scraping: https://imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=1251
Scraping: https://imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=1501
Scraping: https://imdb.com/search/title/?title_type=feature,tv_movie,short&num_votes=50,&adult=include&count=250&start=17

MissingSchema: ignored

In [ ]:
df_movieCorpus = pd.DataFrame(results)
df_movieCorpus

,title,year,plot_summary,director
0,Scream VI,2023,"In the next installment, the survivors of the ...","Matt Bettinelli-Olpin, Tyler Gillett"
1,Creed III,2023,Adonis has been thriving in both his career an...,Michael B. Jordan
2,Everything Everywhere All at Once,2022,A middle-aged Chinese immigrant is swept up in...,"Daniel Kwan, Daniel Scheinert"
3,Cocaine Bear,2023,"An oddball group of cops, criminals, tourists ...",Elizabeth Banks
4,The Whale,2022,"A reclusive, morbidly obese English teacher at...",Darren Aronofsky
...,...,...,...,...
130934,Havar,2009,Sehmus goes mad when he hears rumors about his...,Mehmet Güleryüz
130935,Beder Meye Josna,1991,"The movie is the story of a snake catcher, Jyo...",Tojammel Haque Bokul
130936,Dhadakebaaz,1990,"Lakshya finds a bottle, which has Gangaram tra...",Mahesh Kothare
130937,Kinnara Thumbikal,2000,A young boy falls in love with his older cousi...,R.J. Prasad


In [ ]:
df_movieCorpus.to_csv('Movies_Corpus.csv', encoding = 'utf-8-sig', index = False)